---
# **PREDIÇÃO DE TEMPERATURA**
---


# 📌 1. Entendimento do problema


Objetivo: entender o comportamento da temperatura média diária e identificar padrões ou variáveis associadas úteis para modelagem preditiva.

Contexto:
- Previsão de temperatura é essencial para agricultura, saúde pública e planejamento energético.
- Este projeto foca em prever temperatura diária com base em variáveis meteorológicas passadas (ex: umidade, precipitação, vento).

Referências:
- Modelos comuns: Regressão Linear, Random Forest, SVM, XGBoost, LSTM.
- Features típicas: temperatura defasada, umidade relativa, precipitação, variáveis temporais (dia, mês, estação).





In [ ]:
import sys
from pathlib import Path

# Adiciona a raiz do projeto ao sys.path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Pacotes principais
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from src.features import build_features


sns.set(style="whitegrid")

# 📌 2. Carregamento dos Dados


In [2]:
df = pd.read_csv("../data/raw/nasa_rio_vermelho.csv", skiprows=26, sep=",", encoding="utf-8", engine="python")
df = build_features(df)
df.head()

,year,day_of_year,temperature,humidity,precipitation,wind_speed,irradiance,month,temperature_lag1,temperature_lag3,temperature_lag7,temperature_roll3,humidity_roll3,precipitation_roll7
date,,,,,,,,,,,,,,
2020-01-08,2020,8,24.54,85.94,23.25,5.65,14.69,1,26.06,24.62,25.97,25.356667,84.326667,49.54
2020-01-09,2020,9,26.04,83.84,0.14,2.85,28.47,1,24.54,25.47,25.07,25.546667,85.213333,39.32
2020-01-10,2020,10,26.29,90.91,6.33,4.37,22.46,1,26.04,26.06,24.60,25.623333,86.896667,42.16
2020-01-11,2020,11,26.21,91.71,25.27,6.02,16.31,1,26.29,24.54,24.51,26.180000,88.820000,63.52
2020-01-12,2020,12,25.29,78.31,1.98,6.09,23.39,1,26.21,26.04,24.62,25.930000,86.976667,64.84


# 📌 3. QUALIDADE DOS DADOS

In [3]:
# Verifica tipos, nulos, e duplicatas
df.info()
df.isnull().sum()
df.duplicated().sum()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1820 entries, 2020-01-08 to 2024-12-31
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   year                 1820 non-null   int64  
 1   day_of_year          1820 non-null   int64  
 2   temperature          1820 non-null   float64
 3   humidity             1820 non-null   float64
 4   precipitation        1820 non-null   float64
 5   wind_speed           1820 non-null   float64
 6   irradiance           1820 non-null   float64
 7   month                1820 non-null   int32  
 8   temperature_lag1     1820 non-null   float64
 9   temperature_lag3     1820 non-null   float64
 10  temperature_lag7     1820 non-null   float64
 11  temperature_roll3    1820 non-null   float64
 12  humidity_roll3       1820 non-null   float64
 13  precipitation_roll7  1820 non-null   float64
dtypes: float64(11), int32(1), int64(2)
memory usage: 206.2 KB


np.int64(0)

# 📌 4. ENGENHARIA DE VARIÁVEIS

In [29]:
df.head()

,year,day_of_year,temperature,humidity,precipitation,wind_speed,irradiance,month,temperature_lag1,temperature_lag3,temperature_lag7,temperature_roll3,humidity_roll3,precipitation_roll7,target
date,,,,,,,,,,,,,,,
2020-01-08,2020,8,24.54,85.94,23.25,5.65,14.69,1,26.06,24.62,25.97,25.356667,84.326667,49.54,26.04
2020-01-09,2020,9,26.04,83.84,0.14,2.85,28.47,1,24.54,25.47,25.07,25.546667,85.213333,39.32,26.29
2020-01-10,2020,10,26.29,90.91,6.33,4.37,22.46,1,26.04,26.06,24.60,25.623333,86.896667,42.16,26.21
2020-01-11,2020,11,26.21,91.71,25.27,6.02,16.31,1,26.29,24.54,24.51,26.180000,88.820000,63.52,25.29
2020-01-12,2020,12,25.29,78.31,1.98,6.09,23.39,1,26.21,26.04,24.62,25.930000,86.976667,64.84,24.88


In [30]:
df.to_csv("../data/processed/base_modelagem.csv", index=False)


# 📌 5. ANÁLISE EXPLORATÓRIA

In [ ]:
# Série temporal da temperatura média
plt.figure(figsize=(14, 4))
plt.plot(df["date"], df["temp_mean"])
plt.title("Temperatura Média Diária")
plt.xlabel("Data")
plt.ylabel("Temp (°C)")
plt.tight_layout()
plt.show()

In [ ]:
# Histogramas por mês
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x="month", y="temp_mean")
plt.title("Distribuição da temperatura média por mês")
plt.xlabel("Mês")
plt.ylabel("Temp (°C)")
plt.show()


In [ ]:
# Correlação entre variáveis numéricas
plt.figure(figsize=(10, 8))
corr = df.select_dtypes(include=np.number).corr()
sns.heatmap(corr, annot=True, cmap="coolwarm", fmt=".2f")
plt.title("Mapa de correlação entre variáveis")
plt.show()

# 📌 6. INSIGHTS

- A temperatura apresenta forte sazonalidade anual.
- Há correlação significativa entre temperatura média e variáveis como umidade e precipitação.
- Lags e médias móveis parecem úteis para modelos supervisionados.

Próximos passos:
- Construção de modelos simples para baseline (regressão linear, RF)
- Comparação com modelos não-lineares e sequenciais